In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
import numpy as np
import cvxpy as cp

from evanqp import MPCProblem, Polytope, Verifier
from evanqp.layers import BoundArithmetic

In [3]:
class MPC2(MPCProblem):
    def __init__(self, N=3):
        self.N = N

        n = 3
        m = 1

        # Dynamics
        self.A = np.array([[1.0, 0.5, 0.125], [0.0, 1.0, 0.5], [0.0, 0.0, 1.0]])
        self.B = np.array([[0.02], [0.125], [0.5]])
        self.C = np.eye(n)
        
        # Weights
        self.Q = np.diag([1.0, 1.0, 1.0])
        self.R = np.array([[1.0]])
        self.P = np.zeros((n, n))
        
        # Constraints
        self.x_max = np.array([20.0, 3.0, 1.0])
        self.x_min = np.array([-20.0, -3.0, -1.0])
        self.u_max = np.array([0.5])
        self.u_min = np.array([-0.5])
        
        self.x0 = cp.Parameter(n, name='x0')
        self.x = cp.Variable((N + 1, n), name='x')
        self.u0 = cp.Variable(m, name='u0')
        self.u = cp.Variable((N, m), name='u')

        objective = cp.quad_form(self.x[N, :], self.P)
        constraints = [self.x0 == self.x[0, :], self.u0 == self.u[0, :]]

        for i in range(N):
            objective += cp.quad_form(self.x[i, :], self.Q) + cp.quad_form(self.u[i, :], self.R)
            constraints += [self.x[i + 1, :] == self.A @ self.x[i, :] + self.B @ self.u[i, :]]
            constraints += [self.x_min <= self.x[i, :], self.x[i, :] <= self.x_max]
            constraints += [self.u_min <= self.u[i, :], self.u[i, :] <= self.u_max]
        constraints += [self.x_min <= self.x[N, :], self.x[N, :] <= self.x_max]

        self.objective = cp.Minimize(objective)
        self.prob = cp.Problem(self.objective, constraints)

    def problem(self):
        return self.prob

    def parameters(self):
        return [self.x0]

    def variables(self):
        return [self.u0]

    def solve(self, x0):
        self.x0.value = x0
        try:
            self.prob.solve(solver=cp.GUROBI)
        except:
            pass

        solution = {self.u0: self.u0.value,
                    self.u: self.u.value,
                    self.x: self.x.value,
                    self.objective: self.objective.value}
        return solution
    
    def dynamics(self):
        return self.A, self.B

    def reduced_objective(self):
        objective = cp.quad_form(self.x[self.N, :], self.P)
        for i in range(1, self.N):
            objective += cp.quad_form(self.x[i, :], self.Q) + cp.quad_form(self.u[i, :], self.R)
        return cp.Minimize(objective)

In [4]:
class MPC6(MPCProblem):
    def __init__(self, N=10):
        self.N = N

        n = 2
        m = 2

        # Dynamics
        self.A = np.array([[1.0, 1.0], [0.0, 1.0]])
        self.B = np.array([[0.42, 0.9], [0.38, 0.67]])
        self.C = np.eye(n)
        
        # Weights
        self.Q = np.diag([1.0, 1.0])
        self.R = np.diag([30.0, 30.0])
        self.P = np.zeros((n, n))
        
        # Constraints
        self.x_max = np.array([40.0, 10.0])
        self.x_min = np.array([-40.0, -10.0])
        self.u_max = np.array([0.1, 0.1])
        self.u_min = np.array([-0.1, -0.1])
        
        self.x0 = cp.Parameter(n, name='x0')
        self.x = cp.Variable((N + 1, n), name='x')
        self.u0 = cp.Variable(m, name='u0')
        self.u = cp.Variable((N, m), name='u')

        objective = cp.quad_form(self.x[N, :], self.P)
        constraints = [self.x0 == self.x[0, :], self.u0 == self.u[0, :]]

        for i in range(N):
            objective += cp.quad_form(self.x[i, :], self.Q) + cp.quad_form(self.u[i, :], self.R)
            constraints += [self.x[i + 1, :] == self.A @ self.x[i, :] + self.B @ self.u[i, :]]
            constraints += [self.x_min <= self.x[i, :], self.x[i, :] <= self.x_max]
            constraints += [self.u_min <= self.u[i, :], self.u[i, :] <= self.u_max]
        constraints += [self.x_min <= self.x[N, :], self.x[N, :] <= self.x_max]

        self.objective = cp.Minimize(objective)
        self.prob = cp.Problem(self.objective, constraints)

    def problem(self):
        return self.prob

    def parameters(self):
        return [self.x0]

    def variables(self):
        return [self.u0]

    def solve(self, x0):
        self.x0.value = x0
        try:
            self.prob.solve(solver=cp.GUROBI)
        except:
            pass

        solution = {self.u0: self.u0.value,
                    self.u: self.u.value,
                    self.x: self.x.value,
                    self.objective: self.objective.value}
        return solution
    
    def dynamics(self):
        return self.A, self.B

    def reduced_objective(self):
        objective = cp.quad_form(self.x[self.N, :], self.P)
        for i in range(1, self.N):
            objective += cp.quad_form(self.x[i, :], self.Q) + cp.quad_form(self.u[i, :], self.R)
        return cp.Minimize(objective)

In [5]:
mpc_controller = MPC6()

In [6]:
parameter_set = Polytope(np.vstack((np.eye(2), -np.eye(2))), np.concatenate((mpc_controller.x_max, -mpc_controller.x_min)))
# parameter_set = Polytope(np.vstack((np.eye(3), -np.eye(3))), np.concatenate((mpc_controller.x_max, -mpc_controller.x_min)))

In [42]:
verifier = Verifier(parameter_set, mpc_controller)
verifier.compute_bounds(method=BoundArithmetic.ZONO_ARITHMETIC, dual_bound=1e3)
verifier.compute_bounds_lipschitz(dual_bound=1e3)

QP Gain Bound: 100%|██████████| 2/2 [00:00<00:00, 59.57it/s]


[{'lb': array([-0.1, -0.1]), 'ub': array([0.1, 0.1])}]

In [43]:
lip_constant, gain, parameters = verifier.lipschitz_constant(norm=1)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1408 rows, 827 columns and 3278 nonzeros
Model fingerprint: 0x87f27a9e
Model has 509 general constraints
Variable types: 743 continuous, 84 integer (84 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e+03]
  RHS range        [1e-01, 1e+03]
  GenCon rhs range [1e-01, 4e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 807 rows and 468 columns
Presolve time: 0.07s
Presolved: 601 rows, 359 columns, 1835 nonzeros
Presolved model has 4 SOS constraint(s)
Variable types: 309 continuous, 50 integer (50 binary)

Root relaxation: objective 5.671540e+02, 870 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     

In [44]:
print(f'Lipschitz constant 1-Norm: {lip_constant}')
print(f'Gain (jacobian):')
print(gain)
print(f'at: {parameters}')

Lipschitz constant 1-Norm: 1.7739314159656487
Gain (jacobian):
[[-2.36470015e-01 -1.53746140e+00]
 [-1.42108547e-13 -1.13686838e-13]]
at: [-40.           6.36155444]


In [45]:
lip_constant, gain, parameters = verifier.lipschitz_constant(norm=np.inf)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1408 rows, 827 columns and 3278 nonzeros
Model fingerprint: 0xab5051dd
Model has 509 general constraints
Variable types: 743 continuous, 84 integer (84 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e+03]
  RHS range        [1e-01, 1e+03]
  GenCon rhs range [1e-01, 4e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 810 rows and 471 columns
Presolve time: 0.07s
Presolved: 598 rows, 356 columns, 1829 nonzeros
Presolved model has 4 SOS constraint(s)
Variable types: 309 continuous, 47 integer (47 binary)

Root relaxation: objective 5.563442e+02, 646 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     

In [46]:
print(f'Lipschitz constant Inf-Norm: {lip_constant}')
print(f'Gain (jacobian):')
print(gain)
print(f'at {parameters}')

Lipschitz constant Inf-Norm: 1.537461401237088
Gain (jacobian):
[[-2.36470015e-01 -1.53746140e+00]
 [ 2.27373675e-13  0.00000000e+00]]
at [-16.04532998   2.76599232]
